In [1]:
import os 
import sys 

current_dir = os.getcwd()
dir_to_use = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(dir_to_use)

In [2]:
from premsql.datasets import Text2SQLDataset

# load the bird dataset

bird_dataset = Text2SQLDataset(
    dataset_name='bird', split="train", force_download=False,
    dataset_folder="../data"
)

2024-09-04 00:07:12,991 - [BIRD-DATASET] - INFO - Loaded Bird Dataset


In [3]:
bird_dataset = bird_dataset.setup_dataset(
    model_name_or_path="premai-io/prem-1B-SQL", num_fewshot=3, num_rows=3
)

2024-09-04 00:07:13,391 - [BIRD-DATASET] - INFO - Setting up Bird Dataset
Applying prompt: 100%|██████████| 3/3 [00:00<00:00, 2845.53it/s]
2024-09-04 00:07:14,951 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-04 00:07:14,951 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 3/3 [00:00<00:00, 363.25it/s]


In [4]:
bird_dataset[0]


{
│   'input_ids': tensor([32013, 32013,  2042,  ...,   207,    16, 32021]),
│   'labels': tensor([ -100,  -100,  -100,  ...,   207,    16, 32021]),
│   'raw': {
│   │   'db_id': 'movie_platform',
│   │   'question': 'Name movie titles released in year 1945. Sort the listing by the descending order of movie popularity.',
│   │   'evidence': 'released in the year 1945 refers to movie_release_year = 1945;',
│   │   'SQL': 'SELECT movie_title FROM movies WHERE movie_release_year = 1945 ORDER BY movie_popularity DESC LIMIT 1',
│   │   'db_path': '../data/bird/train/train_databases/movie_platform/movie_platform.sqlite',
│   │   'prompt': '<｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer\n### Instruction:\n\n# Follow these ins

In [5]:
# Loading Spider Dataset

spider_dataset = Text2SQLDataset(
    dataset_name="spider",
    split="train",
    dataset_folder="../data",
    ).setup_dataset(
        num_fewshot=3,
        num_rows=3,
        model_name_or_path="premai-io/prem-1B-SQL",
    )

spider_dataset[0]

2024-09-04 00:07:16,667 - [SPIDER-DATASET] - INFO - Loaded Spider Dataset
2024-09-04 00:07:16,671 - [SPIDER-DATASET] - INFO - Setting up Spider Dataset
Applying prompt: 100%|██████████| 3/3 [00:00<00:00, 312.98it/s]
2024-09-04 00:07:17,050 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-04 00:07:17,050 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 3/3 [00:00<00:00, 716.53it/s]



{
│   'input_ids': tensor([32013, 32013,  2042,   417,   274, 20926, 14244, 20391,    11, 26696,
│   │     254, 20676, 30742,   339,  8589,  2008,    11,  6908,   457, 20676,
│   │   30742,  7958,    11,   285,   340,   885,  3495,  4301,  4512,   276,
│   │    4531,  8214,    13,  1487,  4636,  2223, 13143,  4301,    11,  5411,
│   │     285, 13936,  4447,    11,   285,   746,  2159,    12, 13517,   250,
│   │    8214,  4301,    11,   340,   540, 20857,   276,  3495,   185, 13518,
│   │    3649,  3475,    25,   185,   185,     2, 23114,  1067, 12271,    25,
│   │     185,  2042,   540,   330,  2017,  3947,  5202,   280,  7214,   280,
│   │     245,  4892,    13,  4506,  3100,   317,   276,  3697,  2907,   185,
│   │    4014,  2082,  5975,  5151,  2842,   331,   254,  2503,  3625,    13,
│   │    6456,  1098,  2037,    25,   185,   185,    16,    13,  3119,   441,
│   │     957,  2220,  4686,   429,  1328,   889,  1223,   280,   254,  5151,
│   │      13,   804,  1020,   330,   245,  

In [6]:
## Loading Domains dataset

domains = Text2SQLDataset(
    dataset_name="domains",
    split="train",
    dataset_folder="../data",
).setup_dataset(
    num_fewshot=3,
    num_rows=3,
    model_name_or_path="premai-io/prem-1B-SQL",
)

domains[0]

2024-09-04 00:07:17,129 - [DOMAINS-DATASET] - INFO - Loaded Domains Dataset
2024-09-04 00:07:17,131 - [DOMAINS-DATASET] - INFO - Setting up Domains Dataset
Applying prompt: 100%|██████████| 3/3 [00:00<00:00, 1625.07it/s]
2024-09-04 00:07:17,588 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-04 00:07:17,589 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 3/3 [00:00<00:00, 364.85it/s]



{
│   'input_ids': tensor([32013, 32013,  2042,  ...,   304,    26, 32021]),
│   'labels': tensor([ -100,  -100,  -100,  ...,   304,    26, 32021]),
│   'raw': {
│   │   'db_id': 'Aminer_Simplified',
│   │   'question': 'Show the different keywords articles which has shown at Science and Technology Management Research used.',
│   │   'query': "SELECT DISTINCT Paper_Keywords.keyword FROM Paper_Keywords JOIN (SELECT paper_id FROM Venue_Papers WHERE venue_id = (SELECT id FROM Venue WHERE DisplayName = 'Science and Technology Management Research')) AS STM_Papers ON Paper_Keywords.paper_id = STM_Papers.paper_id;",
│   │   'domain': 'Aminer',
│   │   'SQL': "SELECT DISTINCT Paper_Keywords.keyword FROM Paper_Keywords JOIN (SELECT paper_id FROM Venue_Papers WHERE venue_id = (SELECT id FROM Venue WHERE DisplayName = 'Science and Technology Management Research')) AS STM_Papers ON Paper_Keywords.paper_id = STM_Papers.paper_id;",
│   │   'db_path': '../data/domains/databases/Aminer_Simplified/Ami

In [7]:
# Loading Gretel AI Dataset

gretel_dataset = Text2SQLDataset(
    dataset_name="gretel",
    split="train",
    dataset_folder="../data",
).setup_dataset(
    num_fewshot=3,
    num_rows=3,
    model_name_or_path="premai-io/prem-1B-SQL",
)

gretel_dataset[0]

Applying prompt: 100%|██████████| 3/3 [00:00<00:00, 16491.37it/s]
2024-09-04 00:07:18,922 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-04 00:07:18,923 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 3/3 [00:00<00:00, 1005.35it/s]



{
│   'input_ids': tensor([32013, 32013,  2042,   417,   274, 20926, 14244, 20391,    11, 26696,
│   │     254, 20676, 30742,   339,  8589,  2008,    11,  6908,   457, 20676,
│   │   30742,  7958,    11,   285,   340,   885,  3495,  4301,  4512,   276,
│   │    4531,  8214,    13,  1487,  4636,  2223, 13143,  4301,    11,  5411,
│   │     285, 13936,  4447,    11,   285,   746,  2159,    12, 13517,   250,
│   │    8214,  4301,    11,   340,   540, 20857,   276,  3495,   185, 13518,
│   │    3649,  3475,    25,   185,   185,     2, 23114,  1067, 12271,    25,
│   │     185,  2042,   540,   330,  2017,  3947,  5202,   280,  7214,   280,
│   │     245,  4892,    13,  4506,  3100,   317,   276,  3697,  2907,   185,
│   │    4014,  2082,  5975,  5151,  2842,   331,   254,  2503,  3625,    13,
│   │    6456,  1098,  2037,    25,   185,   185,    16,    13,  3119,   441,
│   │     957,  2220,  4686,   429,  1328,   889,  1223,   280,   254,  5151,
│   │      13,   804,  1020,   330,   245,  

In [10]:
gretel_dataset[0]["raw"]


{
│   'id': 5097,
│   'question': 'What is the total volume of timber sold by each salesperson, sorted by salesperson?',
│   'schema': "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');",
│   'SQL': 'SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;',
│   'context': "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'

In [11]:
# Merge all the datasets

print(f"Length of bird dataset: {len(bird_dataset)}")
print(f"Length of spider dataset: {len(spider_dataset)}")
print(f"Length of domains dataset: {len(domains)}")
print(f"Length of gretel dataset: {len(gretel_dataset)}")

merged_dataset = [*bird_dataset, *spider_dataset, *domains, *gretel_dataset]
print(f"Length of merged dataset: {len(merged_dataset)}")

Length of bird dataset: 3

Length of spider dataset: 3

Length of domains dataset: 3

Length of gretel dataset: 3

Length of merged dataset: 12

In [12]:
merged_dataset[0]


{
│   'input_ids': tensor([32013, 32013,  2042,  ...,   207,    16, 32021]),
│   'labels': tensor([ -100,  -100,  -100,  ...,   207,    16, 32021]),
│   'raw': {
│   │   'db_id': 'movie_platform',
│   │   'question': 'Name movie titles released in year 1945. Sort the listing by the descending order of movie popularity.',
│   │   'evidence': 'released in the year 1945 refers to movie_release_year = 1945;',
│   │   'SQL': 'SELECT movie_title FROM movies WHERE movie_release_year = 1945 ORDER BY movie_popularity DESC LIMIT 1',
│   │   'db_path': '../data/bird/train/train_databases/movie_platform/movie_platform.sqlite',
│   │   'prompt': '<｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer\n### Instruction:\n\n# Follow these ins